<a href="https://colab.research.google.com/github/zekrimohamed/application/blob/main/reflectance_satellite_nourhen_14_07_2023_vfin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies installation

In [ ]:
%%bash
pip install geemap
pip install mercantile ipyleaflet
pip install gdown
pip install rasterio
pip install fiona


In [ ]:
pip install sentinelhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.4/244.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6085 sha256=23243de8e03bdbb4f6f3362153a3a5d179b4c274606719c65026fd3b7344b520
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


In [ ]:
import geemap

import ee
Map = geemap.Map()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=pvUWi3woZHnlNlHi6HEUchPWtxphu0-NN32dFEYBQ8k&tc=eGfH_vTJntIdU-G7d5KD5jcGyB5ff_KgsXOn0nW3avg&cc=5ix5ic041JJ9c95sWx1gKE5eXbfmJMHNNW65_K87T2s

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhU7CRS-ZS1l0K0QysVFaUrRIJ_K53YEv24_kJ2T-yUJLAxUyIIkkbA

Successfully saved authorization token.


#Polygon coordinates

##Polygon using coordinates

In [ ]:
Map.clear()
Map = geemap.Map()
Map.add_basemap("HYBRID")
Map.set_center(lon=8.999290981253452, lat=35.302775191114655, zoom=15)

# Create a polygon geometry using the coordinates
polygon_coords = [[[8.996295, 35.304095], [8.995522, 35.301118], [8.998097, 35.299823], [9.000071, 35.302519], [8.996295, 35.304095]]]
polygon = ee.Geometry.Polygon(polygon_coords)


# Add the polygon to the map
Map.addLayer(polygon, {'color': 'FF0000'}, 'Polygon')

# Display the map
Map


Map(center=[35.302775191114655, 8.999290981253452], controls=(WidgetControl(options=['position', 'transparent_…

## Polygon using shape file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd

In [ ]:
shp_file = gpd.read_file('/content/drive/MyDrive/Sohag/shapefile_sohag/sohag2.shp')

# Get the coordinates of the first polygon in the shapefile
polygon_coords = shp_file['geometry'][0].exterior.coords[:]
# print(polygon_coords)
# Create a list of Earth Engine Points from the coordinates
ee_points = [ee.Geometry.Point(coord) for coord in polygon_coords]
#print(ee_points)
# Create a polygon from the points
polygon = ee.Geometry.Polygon(ee_points)

# Print the polygon
# print(polygon)

## Drawn polygon

In [ ]:
Map.clear()
Map = geemap.Map()
Map.add_basemap("HYBRID")
Map.set_center(lon=8.999290981253452, lat=35.302775191114655, zoom=15)
Map

Map(center=[35.302775191114655, 8.999290981253452], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
polygon = Map.draw_last_feature
if polygon:
  coords = polygon.geometry().getInfo()["coordinates"]
  print(coords)
else:
  print("no polygon drawn yet")

[[[8.996508, 35.303603], [8.995564, 35.300626], [9.000283, 35.297789], [9.001527, 35.300696], [8.996508, 35.303603]]]


## Area measurement

In [ ]:
area = polygon.area().getInfo()
print("Area in m²: ",area)
print("Area in hectares: ",area/10000)

Area in m²:  176253.85401943297
Area in hectares:  17.625385401943298


#Vegetation indices calculation

##First

In [ ]:
# Load Sentinel 2 surface reflectance imagery
image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2023-03-01', '2023-05-20') \
    .filterBounds(polygon) \
    .sort("CLOUD_COVERAGE_ASSESSMENT") \
    .first() \
    .clip(polygon)
# Calculate NDRE
ndre = image.normalizedDifference(['B8', 'B4'])

In [ ]:
# Print the date of the available image
print('Image date:', ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo())


#All

In [ ]:
polygon = ee.Geometry.Polygon(coords)

# Load Sentinel 2 surface reflectance imagery
image= ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2023-01-01', '2023-05-20') \
    .filterBounds(polygon) \
    .sort("CLOUD_COVERAGE_ASSESSMENT")

# Get the list of unique acquisition dates
dates = image.aggregate_array('system:time_start') \
    .map(lambda d: ee.Date(d).format('YYYY-MM-dd')) \
    .distinct() \
    .getInfo()

# Print the available dates
print('Available dates:')
for date in dates:
    print(date)


##Cloud coverage assessment less than 10

In [ ]:
polygon = ee.Geometry.Polygon(coords)

# Load Sentinel 2 surface reflectance imagery
image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2023-01-01', '2023-05-20') \
    .filterBounds(polygon) \
    .filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 10) \
    .sort("CLOUD_COVERAGE_ASSESSMENT")

# Get the list of unique acquisition dates
dates = image.aggregate_array('system:time_start') \
    .map(lambda d: ee.Date(d).format('YYYY-MM-dd')) \
    .distinct() \
    .getInfo()

# Print the available dates
print('Available dates:')
for date in dates:
    print(date)


#NDRE

In [ ]:

# Calculate NDRE
ndre = image.normalizedDifference(['B8', 'B5'])


In [ ]:
# Compute the mean NDRE
mean_ndre = ndre.reduceRegion(
  reducer=ee.Reducer.mean(),
  geometry=polygon,
  scale=10
)

# Print the mean NDRE
print("Mean NDRE:", mean_ndre.getInfo())

# Compute the min NDRE
min_ndre = ndre.reduceRegion(
  reducer=ee.Reducer.min(),
  geometry=polygon,
  scale=10
)
# Print the min NDRE
print("Min NDRE:", min_ndre.getInfo())

# Compute the max NDRE
max_ndre = ndre.reduceRegion(
  reducer=ee.Reducer.max(),
  geometry=polygon,
  scale=10
)

# Print the max NDRE
print("Max NDRE:", max_ndre.getInfo())

In [ ]:
# # Display the NDRE on a map
 # Clear the map before adding a new legend
Map_NDRE.clear()
Map_NDRE = geemap.Map()
Map_NDRE.add_basemap("HYBRID")
Map_NDRE.set_center(lon=8.999290981253452, lat=35.302775191114655, zoom=15)

palette = ['#a50026','#d73027', '#f46d43', '#fdae61', '#fee08b', '#ffffbf' ,'#d9ef8b', '#a6d96a', '#66bd63','#1a9850','#006837']

min = min_ndre.getInfo()
max = max_ndre.getInfo()
min = min["nd"]
max = max["nd"]
num_steps = 11

step = (max - min) / (num_steps - 1)
print(max-min)
interval = []
legend_dict = {}
for i in range(num_steps):
    value = min + i * step
    interval.append(round(value, 2))
    start_value = min + i * step
    end_value = min + (i + 1) * step

for i in range(10):
    start = min + i * step
    end = min + (i + 1) * step
    key = '{:.2f} - {:.2f}'.format(start, end)
    legend_dict[key] = palette[i]
# legend_dict['{:.2f} - {:.2f}'.format(end, end_value)] = palette[13]
print(interval)
print(legend_dict)

ndre_layer = {'palette': palette, 'min': min, 'max':max, 'interval': interval}

Map_NDRE.addLayer(ndre, ndre_layer, 'NDRE')
Map_NDRE.add_legend(title='NDRE', legend_dict=legend_dict)
Map_NDRE.addLayerControl()
Map_NDRE.centerObject(ndre, 15)
Map_NDRE

#NDVI

In [ ]:
# Calculate the NDVI
ndvi = image.normalizedDifference(['B8', 'B4'])

In [ ]:
# Get the mean NDVI value for the area of interest
mean_ndvi = ndvi.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': polygon,
    'scale': 10
}).get('nd')

mean=mean_ndvi.getInfo()
# Print the mean NDVI value
print('Mean NDVI:', mean)

# Get the min NDVI value for the area of interest
min_ndvi = ndvi.reduceRegion(**{
    'reducer': ee.Reducer.min(),
    'geometry': polygon,
    'scale': 10
}).get('nd')

# Print the min NDVI value
min=min_ndvi.getInfo()
print('Min NDVI:', min)

# Get the max NDVI value for the area of interest
max_ndvi = ndvi.reduceRegion(**{
    'reducer': ee.Reducer.max(),
    'geometry': polygon,
    'scale': 10
}).get('nd')

# Print the max NDVI value
max=max_ndvi.getInfo()
print('Max NDVI:', max)

In [ ]:
Map.clear()
Map = geemap.Map()

palette = ['#000000','#a50026','#d73027', '#f46d43', '#fdae61', '#fee08b', '#ffffbf', '#d9ef8b', '#a6d96a', '#66bd63','#1a9850','#006837']

num_steps = 11
step = (max - min) / (num_steps - 1)

interval = []
legend_dict = {}
for i in range(num_steps):
    value = min + i * step
    interval.append(round(value, 2))
    start_value = min + i * step
    end_value = min + (i + 1) * step
legend_dict['{:.2f} - {:.2f}'.format(0, min)] = palette[0]
for i in range(1,10):
    start = min + i * step
    end = min + (i + 1) * step
    key = '{:.2f} - {:.2f}'.format(start, end)
    legend_dict[key] = palette[i]
legend_dict['{:.2f} - {:.2f}'.format(end, 1)] = palette[10]
print(interval)
print(legend_dict)
# Set up the NDVI layer with the customized palette and interval
ndvi_layer = {'palette': palette, 'min': min, 'max': max, 'interval': interval}
Map.add_basemap("SATELLITE")
Map.addLayer(ndvi, ndvi_layer, 'NDVI')
Map.centerObject(ndvi, 16)
Map.add_legend(title='NDVI', legend_dict=legend_dict)
# Display the map
Map

In [ ]:
import geemap
import ee
import pandas as pd

ee.Initialize()

# Clear previous map and create a new one
Map.clear()
Map = geemap.Map()

# Your code for palette, num_steps, step, interval, and legend_dict

# Set up the NDVI layer with the customized palette and interval
ndvi_layer = {'palette': palette, 'min': min, 'max': max, 'interval': interval}
Map.add_basemap("SATELLITE")
Map.addLayer(ndvi, ndvi_layer, 'NDVI')
Map.centerObject(ndvi, 16)
Map.add_legend(title='NDVI', legend_dict=legend_dict)

# Display the map
Map




In [ ]:
# Extract the NDVI values as a list
ndvi_values = ndvi.sample(region=polygon, scale=10).aggregate_array('nd').getInfo()

# Create a DataFrame with NDVI values
df_ndvi = pd.DataFrame(ndvi_values, columns=['NDVI'])

# Save the DataFrame as a CSV file
csv_path = '/content/drive/MyDrive/pfe/Nourhen( image satilitaire)/reflectance_satellite/ndvi_values.csv'
df_ndvi.to_csv(csv_path, index=False)

print("NDVI values DataFrame saved successfully.")


#Reflectance

In [ ]:
polygon = ee.Geometry.Polygon(coords)
# Load Sentinel 2 surface reflectance imagery
image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2023-03-01', '2023-05-20') \
    .filterBounds(polygon) \
    .sort("CLOUD_COVERAGE_ASSESSMENT") \
    .first() \
    .clip(polygon)



In [ ]:

# Clip the images to the polygon
clipped_collection = image_collection.map(lambda image: image.clip(polygon))

# Calculate reflectance for each pixel in the clipped collection
def calculateReflectance(image):
    # Calculate reflectance for each band
    reflectance = image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A','B11','B12']).divide(10000)
    return reflectance

# Apply the calculation to each image in the clipped collection
reflectance_collection = clipped_collection.map(calculateReflectance)
# Get the first image from the collection
first_image = reflectance_collection.first()# Choose the reducer method (e.g., mean, median, first)
#first_image = reflectance_collection.median ()# Choose the reducer method (e.g., mean, median, first)
#first_image = reflectance_collection.mean()# Choose the reducer method (e.g., mean, median, first)
# Get the reflectance values for each pixel within the polygon
reflectance_data = first_image.sample(region=polygon, scale=10)
# Get the reflectance values for a sample pixel
sample_pixel = reflectance_data.first() # Choose the reducer method (e.g., mean, median, first)

# Print reflectance values for each band
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A','B11','B12']
for band in bands:
    reflectance_band = sample_pixel.get(band)
    print(f"{band}: {reflectance_band.getInfo()}")

NameError: ignored

In [ ]:
# Clip the images to the polygon
clipped_collection = image_collection.map(lambda image: image.clip(polygon))

# Calculate reflectance for each pixel in the clipped collection
def calculateReflectance(image):
    # Calculate reflectance for each band
    reflectance = image.select(['B2', 'B3', 'B4', 'B5','B8']).divide(10000)
    return reflectance

# Apply the calculation to each image in the clipped collection
reflectance_collection = clipped_collection.map(calculateReflectance)
# Get the first image from the collection
first_image = reflectance_collection.first()# Choose the reducer method (e.g., mean, median, first)
#first_image = reflectance_collection.median ()# Choose the reducer method (e.g., mean, median, first)
#first_image = reflectance_collection.mean()# Choose the reducer method (e.g., mean, median, first)
# Get the reflectance values for each pixel within the polygon
reflectance_data = first_image.sample(region=polygon, scale=10)
# Get the reflectance values for a sample pixel
sample_pixel = reflectance_data.first() # Choose the reducer method (e.g., mean, median, first)

# Print reflectance values for each band
bands = ['B2', 'B3', 'B4', 'B5','B8']
for band in bands:
    reflectance_band = sample_pixel.get(band)
    print(f"{band}: {reflectance_band.getInfo()}")

NameError: ignored

In [ ]:
polygon = ee.Geometry.Polygon(coords)
# Load Sentinel 2 surface reflectance imagery
image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2023-03-01', '2023-05-20') \
    .filterBounds(polygon) \
    .sort("CLOUD_COVERAGE_ASSESSMENT") \
    .first() \
    .clip(polygon)

In [ ]:

# Clip the images to the polygon
clipped_collection = image_collection.map(lambda image: image.clip(polygon))

# Calculate reflectance for each pixel in the clipped collection
def calculateReflectance(image):
    # Calculate reflectance for each band
    reflectance = image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A','B11','B12']).divide(10000)
    return reflectance

# Apply the calculation to each image in the clipped collection
reflectance_collection = clipped_collection.map(calculateReflectance)
# Get the first image from the collection
first_image = reflectance_collection.first()# Choose the reducer method (e.g., mean, median, first)
#first_image = reflectance_collection.median ()# Choose the reducer method (e.g., mean, median, first)
#first_image = reflectance_collection.mean()# Choose the reducer method (e.g., mean, median, first)
# Get the reflectance values for each pixel within the polygon
reflectance_data = first_image.sample(region=polygon, scale=10)
# Get the reflectance values for a sample pixel
sample_pixel = reflectance_data.first() # Choose the reducer method (e.g., mean, median, first)

# Print reflectance values for each band
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A','B11','B12']
for band in bands:
    reflectance_band = sample_pixel.get(band)
    print(f"{band}: {reflectance_band.getInfo()}")

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
# Get the wavelengths for the bands
wavelengths = [492.3, 558.9, 664.9, 703.8, 739.1, 779.7,832.9,864.0]  # Example wavelengths for bands B2, B3, B4, B8

#wavelengths = [492.3, 558.9, 664.9, 703.8, 739.1, 779.7,832.9,864.0,1610.4,2185.7]  # Example wavelengths for bands B2, B3, B4, B8
# Get the reflectance values for the sample pixel
reflectance_values = [sample_pixel.get(band).getInfo() for band in ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A']]

# Plot the reflectance values versus wavelength
plt.plot(wavelengths, reflectance_values, marker='o')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.title('Reflectance vs Wavelength')
plt.show()

NameError: ignored

#Reflectance

In [ ]:
import ee
import pandas as pd
ee.Initialize()

polygon = ee.Geometry.Polygon(coords)

# Load Sentinel 2 surface reflectance imagery
image_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2023-03-01', '2023-05-20') \
    .filterBounds(polygon) \
    .sort("CLOUD_COVERAGE_ASSESSMENT")

# Clip the images to the polygon
clipped_collection = image_collection.map(lambda image: image.clip(polygon))

# Calculate reflectance for each pixel in the clipped collection
def calculateReflectance(image):
    # Calculate reflectance for each band
    reflectance = image.select(['B2', 'B3', 'B4', 'B5', 'B8']).divide(10000)
    return reflectance

# Apply the calculation to each image in the clipped collection
reflectance_collection = clipped_collection.map(calculateReflectance)

# Get the first image from the collection
first_image = reflectance_collection.first()

# Get the reflectance values for each pixel within the polygon
reflectance_data = first_image.sample(region=polygon, scale=10)

# Create a list to store the reflectance values
reflectance_values = []

# Iterate over each band and extract the reflectance value for each pixel
bands = ['B2', 'B3', 'B4', 'B5', 'B8']
for band in bands:
    reflectance_band = reflectance_data.aggregate_array(band)
    reflectance_values.append(reflectance_band.getInfo())

# Transpose the reflectance values to have one row per pixel
reflectance_values_transposed = zip(*reflectance_values)

# Create a pandas DataFrame
df = pd.DataFrame(reflectance_values_transposed, columns=bands)
df.head()

,B2,B3,B4,B5,B8
0,0.0964,0.1498,0.1930,0.2940,0.3919
1,0.0986,0.1528,0.2121,0.2631,0.3913
2,0.1108,0.1844,0.2336,0.2631,0.3996
3,0.0800,0.1296,0.1628,0.2145,0.3649
4,0.0925,0.1538,0.2127,0.2631,0.3716


In [ ]:
import ee
import pandas as pd
ee.Initialize()

polygon = ee.Geometry.Polygon(coords)

# Load Sentinel 2 surface reflectance imagery
image_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2023-03-01', '2023-05-20') \
    .filterBounds(polygon) \
    .sort("CLOUD_COVERAGE_ASSESSMENT")

# Clip the images to the polygon
clipped_collection = image_collection.map(lambda image: image.clip(polygon))

# Calculate reflectance for each pixel in the clipped collection
def calculateReflectance(image):
    # Calculate reflectance for each band
    reflectance = image.select(['B2', 'B3', 'B4', 'B5', 'B8']).divide(10000)
    return reflectance

# Apply the calculation to each image in the clipped collection
reflectance_collection = clipped_collection.map(calculateReflectance)

# Get the first image from the collection
first_image = reflectance_collection.first()

# Get the reflectance values for each pixel within the polygon
reflectance_data = first_image.sample(region=polygon, scale=10)

# Create a list to store the reflectance values
reflectance_values = []

# Iterate over each band and extract the reflectance value for each pixel
bands = ['B2', 'B3', 'B4', 'B5', 'B8']
for band in bands:
    reflectance_band = reflectance_data.aggregate_array(band)
    reflectance_values.append(reflectance_band.getInfo())

# Transpose the reflectance values to have one row per pixel
reflectance_values_transposed = zip(*reflectance_values)

# Create a pandas DataFrame
df = pd.DataFrame(reflectance_values_transposed, columns=bands)

df.head()


,B2,B3,B4,B5,B8
0,0.0964,0.1498,0.1930,0.2940,0.3919
1,0.0986,0.1528,0.2121,0.2631,0.3913
2,0.1108,0.1844,0.2336,0.2631,0.3996
3,0.0800,0.1296,0.1628,0.2145,0.3649
4,0.0925,0.1538,0.2127,0.2631,0.3716


In [ ]:
import ee
import pandas as pd

ee.Initialize()

polygon = ee.Geometry.Polygon(coords)

# Load Sentinel 2 surface reflectance imagery
image_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2023-03-01', '2023-05-20') \
    .filterBounds(polygon) \
    .sort("CLOUD_COVERAGE_ASSESSMENT")

# Clip the images to the polygon
clipped_collection = image_collection.map(lambda image: image.clip(polygon))

# Calculate reflectance for each pixel in the clipped collection
def calculateReflectance(image):
    # Calculate reflectance for each band
    reflectance = image.select(['B2', 'B3', 'B4', 'B5', 'B8']).divide(10000)
    return reflectance

# Apply the calculation to each image in the clipped collection
reflectance_collection = clipped_collection.map(calculateReflectance)

# Get the first image from the collection
first_image = reflectance_collection.first()

# Get the reflectance values for each pixel within the polygon
reflectance_data = first_image.sample(region=polygon, scale=10)

# Create a list to store the reflectance values
reflectance_values = []

# Iterate over each band and extract the reflectance value for each pixel
bands = ['B2', 'B3', 'B4', 'B5', 'B8']
for band in bands:
    reflectance_band = reflectance_data.aggregate_array(band)
    reflectance_values.append(reflectance_band.getInfo())

# Transpose the reflectance values to have one row per pixel
reflectance_values_transposed = zip(*reflectance_values)

# Create a pandas DataFrame for reflectance values
df = pd.DataFrame(reflectance_values_transposed, columns=bands)

# Calculate NDVI values for each pixel
ndvi_values = first_image.normalizedDifference(['B8', 'B4']).sample(region=polygon, scale=10).aggregate_array('nd').getInfo()

# Add NDVI values to the DataFrame
df['NDVI'] = ndvi_values

df.head()


In [ ]:
# Define the CSV file path
csv_path = '/content/drive/MyDrive/pfe/Nourhen( image satilitaire)/reflectance_satellite/reflectance.csv'

# Save the DataFrame as a CSV file
df.to_csv(csv_path, index=False)


print("CSV file saved successfully.")

##Soil reflectance

In [ ]:
# Filter the pixels based on NDVI range (-1 to 0.3)
soil_df = df[(np.array(ndvi_values) >= -1) & (np.array(ndvi_values) <= 0.3)]

soil_df.head()


In [ ]:
# Save the filtered DataFrame as a CSV file
csv_path = '/content/drive/MyDrive/pfe/Nourhen( image satilitaire)/reflectance_satellite/Soil reflectance values.csv'
soil_df.to_csv(csv_path, index=False)

print("Filtered DataFrame saved as 'Soil reflectance values.csv' successfully.")

##Canopy reflectance

In [ ]:
# Filter the pixels based on NDVI range (-1 to 0.3)
canopy_df = df[(np.array(ndvi_values) > 0.3) & (np.array(ndvi_values) <= 1)]

canopy_df.head()


In [ ]:
# Save the filtered DataFrame with canopy values as a CSV file
csv_path = '/content/drive/MyDrive/pfe/Nourhen( image satilitaire)/reflectance_satellite/Canopy reflectance values.csv'
canopy_df.to_csv(csv_path, index=False)

print("Filtered DataFrame saved as 'Canopy reflectance values.csv' successfully.")

##Reflectance diagrams

In [ ]:
import ee
import matplotlib.pyplot as plt

ee.Initialize()

polygon = ee.Geometry.Polygon(coords)

# Load Sentinel 2 surface reflectance imagery
image_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2023-03-01', '2023-05-20') \
    .filterBounds(polygon) \
    .sort("CLOUD_COVERAGE_ASSESSMENT")

# Clip the images to the polygon
clipped_collection = image_collection.map(lambda image: image.clip(polygon))

# Calculate reflectance for each pixel in the clipped collection
def calculateReflectance(image):
    # Calculate reflectance for each band
    reflectance = image.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B8']).divide(10000)
    return reflectance

# Apply the calculation to each image in the clipped collection
reflectance_collection = clipped_collection.map(calculateReflectance)

# Get the first image from the collection
first_image = reflectance_collection.first()

# Get the reflectance values for each pixel within the polygon
reflectance_data = first_image.sample(region=polygon, scale=10)

# Get the wavelengths for the bands
wavelengths = [442.7, 492.4, 559.8, 664.6, 703.9, 833.0]  # Example wavelengths for bands B1, B2, B3, B4, B5, B8

# Create a list to store the reflectance values
reflectance_values = []

# Iterate over each band and extract the reflectance value for each pixel
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B8']
for band in bands:
    reflectance_band = reflectance_data.aggregate_array(band)
    reflectance_values.append(reflectance_band.getInfo())

# Transpose the reflectance values to have one row per pixel
reflectance_values_transposed = zip(*reflectance_values)

# Iterate over each pixel's reflectance values and plot them
for pixel_reflectance in reflectance_values_transposed:
    plt.plot(wavelengths, pixel_reflectance, marker='o')

plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.title('Reflectance vs Wavelength')
plt.show()


##Mean reflectance

In [ ]:
import ee
import matplotlib.pyplot as plt

ee.Initialize()

polygon = ee.Geometry.Polygon(coords)

# Load Sentinel 2 surface reflectance imagery
image_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2023-03-01', '2023-05-20') \
    .filterBounds(polygon) \
    .sort("CLOUD_COVERAGE_ASSESSMENT")

# Clip the images to the polygon
clipped_collection = image_collection.map(lambda image: image.clip(polygon))

# Calculate reflectance for each pixel in the clipped collection
def calculateReflectance(image):
    # Calculate reflectance for each band
    reflectance = image.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B8']).divide(10000)
    return reflectance

# Apply the calculation to each image in the clipped collection
reflectance_collection = clipped_collection.map(calculateReflectance)

# Get the mean reflectance image
mean_image = reflectance_collection.mean()

# Get the reflectance values for each band within the polygon
reflectance_data = mean_image.sample(region=polygon, scale=10)

# Get the wavelengths for the bands
wavelengths = [442.7, 492.4, 559.8, 664.6, 703.9, 833.0]  # Example wavelengths for bands B1, B2, B3, B4, B5, B8

# Create a list to store the mean reflectance values
mean_reflectance_values = []

# Iterate over each band and extract the mean reflectance value
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B8']
for band in bands:
    reflectance_band = reflectance_data.aggregate_mean(band)
    mean_reflectance_values.append(reflectance_band.getInfo())

# Plot the mean reflectance values versus wavelength
plt.plot(wavelengths, mean_reflectance_values, marker='o')

plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')
plt.title('Mean Reflectance vs Wavelength')
plt.show()


#RGB

In [ ]:
import ee
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import urllib.request
# Initialize Earth Engine
ee.Initialize()
# Load Sentinel-2 surface reflectance imagery
image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2022-10-01', '2023-01-20') \
    .filterBounds(polygon) \
    .sort("CLOUD_COVERAGE_ASSESSMENT") \
    .first() \
    .clip(polygon)
# Select the RGB bands (B4, B3, B2) from the image
rgb = image.select(['B4', 'B3', 'B2'])
# Get the image visualization parameters
vis_params = {
    'min': 0,
    'max': 3000,
    'gamma': 1.5
}
# Create a URL to the image thumbnail for visualization
thumbnail_url = rgb.getThumbURL(vis_params)
# Open the URL and convert to a NumPy array
image_array = np.array(Image.open(urllib.request.urlopen(thumbnail_url)))
# Display the image array
plt.imshow(image_array)
plt.axis('off')
plt.show()